In [6]:
from gmsh_helpers import gmsh_model_to_mesh
import numpy as np
import dolfinx
import gmsh

freq = 100e3
c0 = 1481
omega = 2*np.pi*freq
k0 = omega / c0          # wavenumber
wave_len = c0 / freq     # wavelength
dims = (30e-2, 60e-2)

n_wave = 3
h_elem = wave_len / n_wave

dims = (50e-2, 30e-2)
tx_radius = 15e-2
tx_aperture_radius = 15e-2
alpha_aperture = np.arcsin(tx_aperture_radius / (2*tx_radius))

In [7]:
# Init GMSH
gmsh.initialize()

model = gmsh.model
model.add("TxDomain")

tx_center = model.geo.addPoint(tx_radius*np.cos(alpha_aperture),0,0, meshSize=h_elem, tag=10)
points = []

points.append(model.geo.addPoint(tx_radius*np.cos(-alpha_aperture+np.pi) + tx_radius*np.cos(alpha_aperture), tx_radius*np.sin(-alpha_aperture+np.pi), 0, meshSize=h_elem, tag=0))
points.append(model.geo.addPoint(tx_radius*np.cos(alpha_aperture+np.pi) + tx_radius*np.cos(alpha_aperture), tx_radius*np.sin(alpha_aperture+np.pi), 0, meshSize=h_elem, tag=1))
rect_domain_geom = [[0., -1/2], [1., -1/2], [1., 1/2], [0., 1/2]]
for rect_geom in rect_domain_geom:
    points.append(model.geo.addPoint(*[gg*dd for gg, dd in zip(rect_geom, dims)],0, meshSize=h_elem, tag=len(points)))

lines = []
lines.append(model.geo.addCircleArc(points[0], tx_center, points[1]))
for pt in points[1:]:
    lines.append(model.geo.addLine(points[pt], points[(pt+1)%6]))

# Curveloop and Surface
curveloop = model.geo.addCurveLoop(lines)
domain = model.geo.addPlaneSurface([curveloop])

# This command is mandatory and synchronize CAD with GMSH Model. The less you launch it, the better it is for performance purpose
gmsh.model.geo.synchronize()

# Physical groups
tx = gmsh.model.addPhysicalGroup(1, [lines[0]], 1)
gmsh.model.setPhysicalName(1, tx, "Tx")
domain_boundary = gmsh.model.addPhysicalGroup(1, lines[1:], 2)
gmsh.model.setPhysicalName(1, domain_boundary, "Domain boundary")
ac_domain = gmsh.model.addPhysicalGroup(2, [domain], 3)
gmsh.model.setPhysicalName(2, ac_domain, "Acoustic domain")

# Mesh (2D)
model.mesh.generate(2)
gmsh.write("TxDomain.msh")

# Gmsh to dolfinx mesh
mesh, cell_tags, facet_tags = gmsh_model_to_mesh(model, cell_data=True, facet_data=True, gdim=2)

# Finalize GMSH
gmsh.finalize()

In [31]:
help(facet_tags)

Help on MeshTags_int32 in module dolfinx.cpp.mesh object:

class MeshTags_int32(pybind11_builtins.pybind11_object)
 |  MeshTags object
 |  
 |  Method resolution order:
 |      MeshTags_int32
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(self: dolfinx.cpp.mesh.MeshTags_int32, arg0: dolfinx.cpp.mesh.Mesh, arg1: int, arg2: numpy.ndarray[numpy.int32], arg3: numpy.ndarray[numpy.int32]) -> None
 |  
 |  ufl_id(...)
 |      ufl_id(self: dolfinx.cpp.mesh.MeshTags_int32) -> int
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  dim
 |  
 |  indices
 |  
 |  mesh
 |  
 |  values
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  name
 |  
 |  ----------------------------------------------------------------------
 |  Static methods inherited from pybind11_buil

In [30]:
# Previous and current solution
u1= interpolate(Constant(0.0), V)
u0= interpolate(Constant(0.0), V)

AttributeError: 'dolfinx.cpp.mesh.MeshTags_int32' object has no attribute 'keys'

In [25]:
V = dolfinx.FunctionSpace(mesh, ("Lagrange", 2))

In [27]:
# Create facet to cell connectivity required to determine boundary facets
boundary_dim = mesh.topology.dim - 1
mesh.topology.create_connectivity(boundary_dim, mesh.topology.dim)
boundary_facets = np.where(np.array(dolfinx.cpp.mesh.compute_boundary_facets(mesh.topology)) == 1)[0]

In [29]:
boundary_dofs = dolfinx.fem.locate_dofs_topological(V, boundary_dim, boundary_facets)
bc = dolfinx.DirichletBC(uD, boundary_dofs)